In [40]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

batch_size=12
learning_rate=0.002
num_epoch=60

In [41]:
#cifar_train=dset.CIFAR10("CIFAR10/", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
#cifar_train=dset.CIFAR10("CIFAR10/", train=True, transform=transforms.Compose([    #Data augmentation
#                                                                               transforms.Scale(36),
#                                                                               transforms.CenterCrop(32),
#                                                                               transforms.RandomHorizontalFlip(),
#                                                                               transforms.Lambda(lambda x: x.rotate(90)),
#                                                                               transforms.ToTensor()
#]))
#cifar_test=dset.CIFAR10("CIFAR10/", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

cifar_train=dset.CIFAR10("CIFAR10/", train=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),]), target_transform=None, download=False)
cifar_test=dset.CIFAR10("CIFAR10/", train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),]), target_transform=None, download=False)

In [42]:
print "cifar_train 길이: ", len(cifar_train)
print "cifar_test 길이: ", len(cifar_test)

image, label = cifar_train.__getitem__(1)
print "image data 형태: ", image.size()
print "label: ", label

img = image.numpy()

r, g, b = img[0,:,:], img[1,:,:], img[2,:,:]
img2 = np.zeros((img.shape[1], img.shape[2], img.shape[0]))
img2[:,:,0], img2[:,:,1], img2[:,:,2] = r, g, b

plt.title("label: %d" %label )
plt.imshow(img2, interpolation='bicubic')
plt.show()

SyntaxError: ignored

In [43]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for j, [imgs, labels] in enumerate(dloader):
            img = imgs.cuda()
            label = Variable(labels).cuda()
        
            output = imodel.forward(img)
            _, output_index = torch.max(output, 1)
        
            total += label.size(0)
            correct += (output_index == label).sum().float()
    print("Accuracy of Test Data: {}".format(100*correct/total))

In [44]:
train_loader=torch.utils.data.DataLoader(list(cifar_train)[:], batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader=torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer=nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            #nn.ReLU(),
            nn.ELU(),
            #nn.LeakyReLU(0.01),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,32,3,padding=1),
            #nn.ReLU(),
            nn.ELU(),
            #nn.LeakyReLU(0.01),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            #nn.ReLU(),
            nn.ELU(),
            #nn.LeakyReLU(0.01),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer=nn.Sequential(
            nn.Linear(64*8*8, 100),
            #nn.ReLU(),
            nn.ELU(),
            #nn.LeakyReLU(0.01),
            nn.Dropout2d(0.2),
            nn.BatchNorm1d(100),
            nn.Linear(100,10)
        )
        for m in self.modules():
          if isinstance(m, nn.Conv2d):
            init.kaiming_normal(m.weight.data)
            m.bias.data.fill_(0)
          if isinstance(m, nn.Linear):
            init.kaiming_normal(m.weight.data)
            m.bias.data.fill_(0)
    
    def forward(self,x):
            out=self.layer(x)
            out=out.view(batch_size,-1)
            out=self.fc_layer(out)
            
            return out
model=CNN().cuda()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


In [45]:
loss_func=nn.CrossEntropyLoss()
#optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model.train()

for i in range(num_epoch):
    for j,[image, label] in enumerate(train_loader):
        x=Variable(image).cuda()
        y_=Variable(label).cuda()
        
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j%1000==0:
            print(j,loss)

0 tensor(2.2739, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(1.7723, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(1.2607, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(1.1730, device='cuda:0', grad_fn=<NllLossBackward>)
4000 tensor(1.3111, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(2.0059, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(1.7114, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(1.2874, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.9711, device='cuda:0', grad_fn=<NllLossBackward>)
4000 tensor(0.9344, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.6015, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(1.5987, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.7813, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.9592, device='cuda:0', grad_fn=<NllLossBackward>)
4000 tensor(0.8686, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(1.1682, device='cuda:0', grad_fn=<NllLos

In [46]:
model.eval()
ComputeAccr(test_loader, model)

Accuracy of Test Data: 76.88075256347656


In [47]:
#netname  = '/content/gdrive/My Drive/Colab Notebooks/CIFAR10.pkl'
torch.save(model, '/content/gdrive/My Drive/Colab Notebooks/CIFAR10.pkl', )

In [17]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive
